# Multilabel Text Classification - Auto Tag Stack Exchange Questions
* Notebook by Adam Lang
* Date: 8/9/2024

# Overview
* In this notebook we will implement Multilabel Text Classification in Machine Learning on a dataset of Stack Exchange questions.
* The dataset has over 85,000 questions.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Workflow
1. Load Data
2. Text Cleaning + Preprocessing
3. Merge Tags with Questions
  * 2 Datasets
     * 1 with Tags
     * 1 with Questions
4. Dataset Preparation
5. Feature Engineering with TF-IDF
6. Model Building
  * Naive Bayes
  * Logistic Regression
  * Model Building Summary
7. Build a Final Question Tagging Pipeline

In [2]:
## imports

# string matching
import re

# data handling
import pandas as pd
import numpy as np

# html data
from bs4 import BeautifulSoup

# NLP tasks in spacy
import spacy
nlp = spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])

pd.set_option('display.max_colwidth', 200)

## Loading Data

In [3]:
data_path = '/content/drive/MyDrive/Colab Notebooks/Classical NLP/Questions.csv'

In [4]:
## load questions dataset
questions_df = pd.read_csv(data_path, encoding='latin-1')

# shape of data
print(f"Shape of questions data: {questions_df.shape}")

# data head
questions_df.head()

Shape of questions data: (85085, 6)


,Id,OwnerUserId,CreationDate,Score,Title,Body
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learning?,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach..."
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain English,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values in statistical tests?,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests...."
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not mean causation,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ..."


### Metadata Info
* We can see there are 85,000 questions in the data above.
* The metadata information:
  * Id: Question ID
  * OwnerUserid: User ID
  * CreationDate: Date of posting question
  * Score: Count of Upvotes received by the question.
  * Title: Title of the question
  * Body: Text body of the question
     * Note: We can see a significant amount of html tags in the Body column which we will have to pre-process.

In [5]:
## load Tags dataset
tags_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Classical NLP/Tags.csv')

## shape of data
print(f"Shape of Tags data: {tags_df.shape}")

## data head
tags_df.head()

Shape of Tags data: (244228, 2)


,Id,Tag
0,1,bayesian
1,1,prior
2,1,elicitation
3,2,distributions
4,2,normality


There are 244,228 tags for 85,000 questions.

## Text Cleaning and Pre-processing

We can define a function to clean the text data.

In [6]:
## function to clean/pre-process data
def cleaner(text):

  # remove HTML tags
  text = BeautifulSoup(text).get_text()

  # retrieve alphabetic characters
  text = re.sub("[^a-zA-Z]", " ", text)

  # lowercase text
  text = text.lower()

  # remove extra spaces
  text = re.sub("[\s]+", " ", text)

  # create spacy doc object
  doc = nlp(text)

  # remove stopwords and lemmatize text
  tokens = [token.lemma_ for token in doc if (token.is_stop==False)]

  # join text back together
  return " ".join(tokens)

In [7]:
## apply function to pre-process text
questions_df['cleaned_text'] = questions_df['Body'].apply(cleaner)

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [8]:
## view index of Body Text pre-clean
questions_df['Body'][1]

"<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\nareas are a lot larger than condensed\nurban areas. Is there a need to account for the area size difference?</li>\n<li>if let's say I have census data\ndating back to 4 - 5 census periods,\nhow far can i forecast it into the\nfuture?</li>\n<li>if some of the census zone change\nlightly in boundaries, how can i\naccount for that change?</li>\n<li>What are the methods to validate\ncensus forecasts? for example, if i\nhave data for existing 5 census\nperiods, should I model the first 3\nand test it on the latter two? or is\nthere another way?</li>\n<li>what's the state of practice in\nforecasting census data, and what are\nsome of the state of the art methods?</li>\n</ul>\n"

In [9]:
## view index of cleaned_text post-clean
questions_df['cleaned_text'][1]

'ways forecast demographic census validation calibration techniques concerns census blocks vary sizes rural areas lot larger condensed urban areas need account area size difference let s census data dating census periods far forecast future census zone change lightly boundaries account change methods validate census forecasts example data existing census periods model test way s state practice forecasting census data state art methods'

## Merge Tags with Questions

Now we can explore the tags data.

In [10]:
# head of tags_df
tags_df.head()

,Id,Tag
0,1,bayesian
1,1,prior
2,1,elicitation
3,2,distributions
4,2,normality


In [11]:
# count unique tags
len(tags_df['Tag'].unique())

1315

In [12]:
# value_counts of Tags
tags_df['Tag'].value_counts()

,count
Tag,
r,13236
regression,10959
machine-learning,6089
time-series,5559
probability,4217
...,...
fmincon,1
netflix-prize,1
american-community-survey,1


In [12]:
# lets remove "-" from tags
tags_df['Tag'] = tags_df['Tag'].apply(lambda x: re.sub("-", " ", x))

Before we can merge the Tags and Questions, we neeed to group the Tags by Id

In [13]:
# group tags by ID
tags_df = tags_df.groupby('Id').apply(lambda x: x['Tag'].values).reset_index(name='tags')

# view df
tags_df.head()

,Id,tags
0,1,"[bayesian, prior, elicitation]"
1,2,"[distributions, normality]"
2,3,"[software, open-source]"
3,4,"[distributions, statistical-significance]"
4,6,[machine-learning]


In [14]:
# now merge tags + questions
df = pd.merge(questions_df, tags_df, how='inner', on='Id')

In [15]:
## setup columns
df = df[['Id', 'Body', 'cleaned_text', 'tags']]

# print shape
print(f"Shape of merged df: {df.shape}")

# head of df
df.head()

Shape of merged df: (85085, 4)


,Id,Body,cleaned_text,tags
0,6,"<p>Last year, I read a blog post from <a href=""http://anyall.org/"">Brendan O'Connor</a> entitled <a href=""http://anyall.org/blog/2008/12/statistics-vs-machine-learning-fight/"">""Statistics vs. Mach...",year read blog post brendan o connor entitled statistics vs machine learning fight discussed differences fields andrew gelman responded favorably simon blomberg r s fortunes package paraphrase pro...,[machine-learning]
1,21,<p>What are some of the ways to forecast demographic census with some validation and calibration techniques?</p>\n\n<p>Some of the concerns:</p>\n\n<ul>\n<li>Census blocks vary in sizes as rural\n...,ways forecast demographic census validation calibration techniques concerns census blocks vary sizes rural areas lot larger condensed urban areas need account area size difference let s census dat...,"[forecasting, population, census]"
2,22,<p>How would you describe in plain English the characteristics that distinguish Bayesian from Frequentist reasoning?</p>\n,describe plain english characteristics distinguish bayesian frequentist reasoning,"[bayesian, frequentist]"
3,31,"<p>After taking a statistics course and then trying to help fellow students, I noticed one subject that inspires much head-desk banging is interpreting the results of statistical hypothesis tests....",taking statistics course trying help fellow students noticed subject inspires head desk banging interpreting results statistical hypothesis tests students easily learn perform calculations require...,"[hypothesis-testing, t-test, p-value, interpretation, intuition]"
4,36,"<p>There is an old saying: ""Correlation does not mean causation"". When I teach, I tend to use the following standard examples to illustrate this point:</p>\n\n<ol>\n<li>number of storks and birth ...",old saying correlation mean causation teach tend use following standard examples illustrate point number storks birth rate denmark number priests america alcoholism start th century noted strong c...,"[correlation, teaching]"


We now have > 85,000 unique questions and over 1300 tags.

## Dataset Preparation

In [17]:
# first we need to get the frequency occurrence of each tag
freq = {}
for i in df['tags']:
  for j in i:
    if j in freq.keys():
      freq[j] = freq[j] + 1
    else:
      freq[j] = 1

Now we can find the most frequent tags.

In [18]:
# sort dictionary descending order
freq = dict(sorted(freq.items(), key=lambda x: x[1], reverse=True))

In [19]:
#get sorted frequency
freq.items()

dict_items([('r', 13236), ('regression', 10959), ('machine-learning', 6089), ('time-series', 5559), ('probability', 4217), ('hypothesis-testing', 3869), ('self-study', 3732), ('distributions', 3501), ('logistic', 3316), ('classification', 2881), ('correlation', 2871), ('statistical-significance', 2666), ('bayesian', 2656), ('anova', 2505), ('normal-distribution', 2181), ('multiple-regression', 2054), ('mixed-model', 1998), ('clustering', 1952), ('neural-networks', 1897), ('mathematical-statistics', 1888), ('confidence-interval', 1776), ('categorical-data', 1703), ('generalized-linear-model', 1614), ('variance', 1576), ('data-visualization', 1549), ('estimation', 1533), ('forecasting', 1422), ('t-test', 1418), ('pca', 1395), ('sampling', 1363), ('cross-validation', 1344), ('repeated-measures', 1335), ('spss', 1296), ('svm', 1283), ('chi-squared', 1261), ('maximum-likelihood', 1209), ('predictive-models', 1189), ('multivariate-analysis', 1116), ('survival', 1081), ('references', 1076), (

In [20]:
## what are the top 10 most frequent tags?
common_tags = list(freq.keys())[:10]
common_tags

['r',
 'regression',
 'machine-learning',
 'time-series',
 'probability',
 'hypothesis-testing',
 'self-study',
 'distributions',
 'logistic',
 'classification']

We will only be using the questions that have the top 10 tags.

This is because "rare" appearing tags will not help the model learn/train.

In [21]:
## extracting questions from top 10 tags
x = []
y = []

# loop over tags
for i in range(len(df['tags'])):

  temp = []
  for j in df['tags'][i]:
    if j in common_tags:
      temp.append(j)

  if(len(temp)>1):
    x.append(df['cleaned_text'][i])
    y.append(temp)

In [22]:
# number of questions left
len(x)

11106

In [23]:
## sample questions
x[:5]

['recently started working tuberculosis clinic meet periodically discuss number tb cases currently treating number tests administered etc d like start modeling counts guessing unusual unfortunately ve little training time series exposure models continuous data stock prices large numbers counts influenza deal cases month mean median var distributed like image lost mists time image eaten grue ve found articles address models like d greatly appreciate hearing suggestions approaches r packages use implement approaches edit mbq s answer forced think carefully m asking got hung monthly counts lost actual focus question d like know fairly visible decline onward reflect downward trend overall number cases looks like number cases monthly reflects stable process maybe seasonality overall stable present looks like process changing overall number cases declining monthly counts wobble randomness seasonality test s real change process identify decline use trend seasonality estimate number cases upco

In [24]:
# sample tags
y[:5]

[['r', 'time-series'],
 ['regression', 'distributions'],
 ['distributions', 'probability', 'hypothesis-testing'],
 ['hypothesis-testing', 'self-study'],
 ['r', 'regression', 'time-series']]

Since this is a multilabel classification problem, we need to transform the target variable y.

To do this we will use the MultiLabelBinarizer from sklearn.

In [25]:
from sklearn.preprocessing import MultiLabelBinarizer
#instantiate binarizer
mlb = MultiLabelBinarizer()

## fit_transform --> get labels
y = mlb.fit_transform(y)
y.shape

(11106, 10)

In [26]:
## sample
y[0,:]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1])

In [27]:
# classes of the multilabel binarizer
mlb.classes_

array(['classification', 'distributions', 'hypothesis-testing',
       'logistic', 'machine-learning', 'probability', 'r', 'regression',
       'self-study', 'time-series'], dtype=object)

Now we are able to split dataset into traditional train and validation sets.

In [28]:
from sklearn.model_selection import train_test_split

# setup train and validation sets
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)

In [29]:
## print the train and val sets
print(f"X_train len is: {len(X_train)}")
print()
print(f"X_val len is: {len(X_val)}")
print()
print(f"y_train len is: {len(y_train)}")
print()
print(f"y_val len is: {len(y_val)}")

X_train len is: 8884

X_val len is: 2222

y_train len is: 8884

y_val len is: 2222


## Feature Engineering using TF-IDF
* This is the "classical" way to do this.
* We could certainly create vector embeddings using SentenceTransformers for a more modern approach, but we will use TF-IDF for standard technique.

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
## init TFIDF --> max size of 5000
word_vec = TfidfVectorizer(max_features=5000)

# fit vectorizer on Train data
word_vec.fit(X_train)

# create TF-IDF vectors for Train set
train_word_features = word_vec.transform(X_train)

In [35]:
## lets view the train_word_features
train_word_features

<8884x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 401787 stored elements in Compressed Sparse Row format>

We now have a sparse matrix.
  * 8,884 questions
  * 5,000 features

In [36]:
## now create TF-IDF vectors for Test/Validation set
test_word_features = word_vec.transform(X_val)
test_word_features

<2222x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 100610 stored elements in Compressed Sparse Row format>

We now have a sparse test matrix
   * 2,222 questions
   * 5,000 features

## Build Machine Learning Models

### 1. Naive Bayes - MultinomialNB

In [37]:
#imports
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

Since a question can have multiple labels, we will use the `OneVsRestClassifier`.
  * This helps us save time when training multiple models at once.
  * The OvR also parallelizes the code making the models run faster as well.

One-vs-rest (OvR for short, also referred to as One-vs-All or OvA) is a heuristic method for using binary classification algorithms for multi-class classification.

It involves splitting the multi-class dataset into multiple binary classification problems.
   * A binary classifier is then trained on each binary classification problem and predictions are made using the model that is the most confident.

For example, given a multi-class classification problem with examples for each class ‘red,’ ‘blue,’ and ‘green‘. This could be divided into three binary classification datasets as follows:

  * Binary Classification Problem 1: red vs [blue, green]
  * Binary Classification Problem 2: blue vs [red, green]
  * Binary Classification Problem 3: green vs [red, blue]

A possible downside of this approach is that it **requires one model to be created for each class.**
   * For example, three classes requires three models.
   * This could be an issue for:
      * large datasets (e.g. millions of rows), * slow models (e.g. neural networks), or
      * very large numbers of classes (e.g. hundreds of classes)

This approach requires that each model predicts a class membership probability or a probability-like score.
  * The argmax of these scores (class index with the largest score) is then used to predict a class.

Source: https://machinelearningmastery.com/one-vs-rest-and-one-vs-one-for-multi-class-classification/



In [38]:
## define the model
nb_model = OneVsRestClassifier(MultinomialNB())

In [39]:
# train model
nb_model.fit(train_word_features, y_train)

OneVsRestClassifier(estimator=MultinomialNB())

In [40]:
# make predictions on train set
train_pred_nb = nb_model.predict_proba(train_word_features)

In [41]:
## view predictions
train_pred_nb[:5]

array([[0.00537031, 0.02158368, 0.02940967, 0.01889877, 0.02088131,
        0.01230726, 0.64319103, 0.69425749, 0.03431985, 0.04530027],
       [0.01916906, 0.0127335 , 0.04449999, 0.14621842, 0.02921426,
        0.01538335, 0.56831804, 0.63936001, 0.02154131, 0.06418146],
       [0.04734991, 0.15280241, 0.17844401, 0.05062134, 0.08731576,
        0.06065591, 0.42385244, 0.17573661, 0.03356571, 0.06420911],
       [0.00377573, 0.00875633, 0.07050085, 0.82842411, 0.00466239,
        0.00859505, 0.75207551, 0.8483481 , 0.01382064, 0.00699011],
       [0.00232004, 0.00700268, 0.03307907, 0.02169856, 0.00974026,
        0.00209056, 0.79231659, 0.42939465, 0.01291017, 0.67787124]])

Summary:
* The predictions we get are in terms of the probabilities for each of the 10 tags or labels.
  * We are not able to calculate the f1 score using these probabilities. We need labels for that!
* So, we need to have a threshold value to convert these probabilities to 0 or 1.
* We thus need to specify a threshold value for this.
* We will select the threshold value that performs best for the train set.
* We can start to see why deep learning may be a more optimal approach to this type of problem, but we can also see how it is important to understand this type of classification problem using classical machine learning algorithms.

In [42]:
## function to convert probs into classes or tags based on threshold value
def classify(pred_prob, threshold):
  y_pred_seq = []

  for i in pred_prob:
    temp=[]
    for j in i:
      if j>=threshold:
        temp.append(1)
      else:
        temp.append(0)
    y_pred_seq.append(temp)

  return y_pred_seq

In [46]:
## function for finding optimal value of threshold
def optimal_threshold(actual, pred_prob):
  # define threshold values
  thresholds = np.arange(0, 0.5, 0.01)

  score = []
  for value in thresholds:
    # get classes for each threshold
    pred_classes = classify(pred_prob,value)
    # get F1-score for every threshold
    score.append(f1_score(actual, pred_classes, average='weighted'))

  return thresholds[score.index(max(score))]

In [47]:
## now find the optimal threshold value
print(f"The Optimal threshold value: {optimal_threshold(y_train, train_pred_nb)}")

The Optimal threshold value: 0.27


In [49]:
## getting clases using optimal threshold
train_pred_nb_class = classify(train_pred_nb, 0.27)

In [50]:
## output
train_pred_nb_class[:5]

[[0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1]]

Above, we can see the labels.

In [51]:
## inverse_transform
mlb.inverse_transform(np.array(train_pred_nb_class[:5]))

[('r', 'regression'),
 ('r', 'regression'),
 ('r',),
 ('logistic', 'r', 'regression'),
 ('r', 'regression', 'time-series')]

Above, we can see the actual tags that go with the labels.

Now we can evaluate on the train set.

In [52]:
## evaluate on train set - in sample prediction
print(f"F1-score on Train set: {f1_score(y_train, train_pred_nb_class, average='weighted')}")

F1-score on Train set: 0.7468370707407613


In [53]:
# make predictions on validation set - out of sample prediction
val_pred_nb = nb_model.predict_proba(test_word_features)

# get the classes
val_pred_nb_class = classify(val_pred_nb, 0.27)

# Evaluate on validation set
print(f"F1-score on Validation Set: {f1_score(y_val, val_pred_nb_class, average='weighted')}")

F1-score on Validation Set: 0.6820040639074911


Summary:
* We can see the F1 score was higher on the training data at 0.74 and lower on the validation/test data at 0.68.
* Let's try a Logistic Regression model and see if we can improve this.

## Logistic Regression

In [54]:
from sklearn.linear_model import LogisticRegression

In [55]:
# define model
log_reg_model = OneVsRestClassifier(LogisticRegression())

# train model
log_reg_model.fit(train_word_features, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [56]:
## make predictions on Train set
train_pred_lr = log_reg_model.predict_proba(train_word_features)

In [57]:
## lets see results
train_pred_lr[:5]

array([[0.02648403, 0.05597941, 0.0708472 , 0.02951809, 0.07055119,
        0.01359619, 0.80032701, 0.90344763, 0.04127528, 0.06075854],
       [0.05794708, 0.03461913, 0.11731043, 0.12959178, 0.06727114,
        0.04589401, 0.62469362, 0.71831553, 0.05851563, 0.12315661],
       [0.0858301 , 0.25219549, 0.35728096, 0.05853359, 0.17848746,
        0.09671004, 0.4998142 , 0.23757032, 0.0696606 , 0.10672391],
       [0.01829057, 0.03623374, 0.16689767, 0.88429219, 0.0137612 ,
        0.02886943, 0.64191141, 0.56548849, 0.03059469, 0.01584995],
       [0.0064751 , 0.01355925, 0.29743964, 0.01964765, 0.0249428 ,
        0.00252226, 0.86122303, 0.32877981, 0.01697299, 0.93965335]])

In [58]:
## find optimal value
print(f"Optimal threshold is: {optimal_threshold(y_train, train_pred_lr)}")

Optimal threshold is: 0.32


In [59]:
# get classes using optimal threshold
train_pred_lr_class = classify(train_pred_lr, 0.32)
train_pred_lr_class[:5]

[[0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1]]

Above we see the logistic regression model labels.

In [60]:
# eval on train set - in sample prediction
print(f"F1-score on Train Set: {f1_score(y_train, train_pred_lr_class, average='weighted')}")

F1-score on Train Set: 0.8270419004256235


In [62]:
# make predictions on validation set - out of sample prediction
val_pred_lr = log_reg_model.predict_proba(test_word_features)

# get classes
val_pred_lr_class = classify(val_pred_lr, 0.32)

# evaluate on validation set
print(f"F1-score on Validation Set: {f1_score(y_val, val_pred_lr_class, average='weighted')}")

F1-score on Validation Set: 0.753221883805125


# Model Building Summary
* We can see the comparision of the models below

`|Model_Name          | Train Set | ValidationSet|`

`|Naive Bayes         |  0.746    | 0.682        |`

`|Logistic Regression  |  0.827   | 0.7532     |`

Thus we can see that Logistic Regression is the better performing model overall.

# Final Question Tagging Inference Pipeline

In [64]:
## create function for inference pipeline
def tagger(question):
  # clean text
  cleaned_question = cleaner(question)

  # feature engineering
  vector = word_vec.transform([cleaned_question])

  # predicting probabilities using logistic regression model (best model)
  pred_prob = log_reg_model.predict_proba(vector)

  # convert probabilities to classes
  pred_class = classify(pred_prob, 0.32)

  #return the transformed class tags
  return mlb.inverse_transform(np.array(pred_class))

## Sample Question


In [70]:
# lets get a random sample question from the original question set
df.Body.sample(1)

,Body
5128,"<p>I am aware of the theory of stochastic gradient descent, which is a faster way of developing linear regression. Through this we can have an '<em>optimized implementation</em>' of linear regress..."


In [72]:
## lets import warnings - we will see errors from using spacy
import warnings
warnings.filterwarnings('ignore')

In [73]:
tagger("<p>I am aware of the theory of stochastic gradient descent, which is a faster way of developing linear regression. Through this we can have an '<em>optimized implementation</em>' of linear regress...")

[('machine-learning', 'regression')]

# Summary
* We can see we were able to assign multiple labels to the text based on the classification model that we built.